In [ ]:
# Tissue Spatial Geometrics Lab

# The "Rotate Green Line" Tool
# David H Nguyen, PhD. 

# This version should print the name of the file that caused an error, so you know which file it was. 

### Instructions ###
# 1. Input files MUST be in PAIRS. 1) The original image, 2) a copy of the original image. 
#   The copy image must have the characters " copy" or " Copy" at the end of its file name (but before the filetype suffix).
# 2. The copy image must have a bright green line in it. The original image cannot have green color in it. 
# 3. The rotated images will appear in the same directory that contains the input images. 
#    Rotated images will have the suffix "rotated" at the end of their file name.


In [ ]:
# Load the dependencies
import os
import cv2
import numpy as np

In [ ]:
# Step 1 - Set Working Directory 
os.chdir('pasteFilePathHere')
path = os.getcwd()

# Create output folder
output_folder = os.path.join(path, "rotated images")
os.makedirs(output_folder, exist_ok=True)

contents = os.listdir(path)
# Remove hidden files (Mac-related)
contents = [item for item in contents if not item.startswith(".")]

contents

In [ ]:


for file_name in contents:
    
    name, ext = os.path.splitext(file_name)

    # Only process files that END with " Copy" or " copy"
    if ext.lower() in valid_extensions and (name.endswith(" Copy") or name.endswith(" copy")):

        try:
            copy_img_path = os.path.join(path, file_name)
            copy_img = cv2.imread(copy_img_path)

            if copy_img is None:
                raise ValueError(f"Error loading copy image: {file_name}")

            # Detect green line on COPY image
            hsv = cv2.cvtColor(copy_img, cv2.COLOR_BGR2HSV)
            
            lower_green = np.array([30, 40, 40])
            upper_green = np.array([90, 255, 255])
            
            mask = cv2.inRange(hsv, lower_green, upper_green)
            
            lines = cv2.HoughLinesP(
                mask,
                1,
                np.pi/180,
                threshold=50,
                minLineLength=10,
                maxLineGap=10
            )
            
            if lines is None:
                raise ValueError(f"No green lines detected in file: {file_name}")

            x1, y1, x2, y2 = lines[0][0]
            angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi

            # Determine matching ORIGINAL filename
            if name.endswith(" - Copy"):
                original_name = name[:-len(" - Copy")]
            elif name.endswith(" - copy"):
                original_name = name[:-len(" - copy")]
            elif name.endswith(" Copy"):
                original_name = name[:-len(" Copy")]
            elif name.endswith(" copy"):
                original_name = name[:-len(" copy")]
            else:
                continue

            original_file = original_name + ext
            original_img_path = os.path.join(path, original_file)

            if not os.path.exists(original_img_path):
                raise ValueError(f"Matching original not found: {original_file}")

            original_img = cv2.imread(original_img_path)

            if original_img is None:
                raise ValueError(f"Error loading original image: {original_file}")

            # Rotate ORIGINAL image
            M = cv2.getRotationMatrix2D(
                (original_img.shape[1] // 2, original_img.shape[0] // 2),
                angle,
                1
            )
            
            rotated_img = cv2.warpAffine(
                original_img,
                M,
                (original_img.shape[1], original_img.shape[0]),
                borderMode=cv2.BORDER_CONSTANT,
                borderValue=(255, 255, 255)
            )

            # Save rotated ORIGINAL into output folder
            output_path = os.path.join(output_folder, original_name + "_rotated" + ext)
            cv2.imwrite(output_path, rotated_img)

            print(f"Rotated original: {original_file}")

        except Exception as e:
            print(f"Error processing file {file_name}: {e}")